In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from statsmodels.graphics import tsaplots
import statsmodels.api as sm
from pylab import rcParams
from scipy.stats.stats import pearsonr
from scipy.stats.stats import spearmanr
import seaborn as sns
import statsmodels.api as sm

In [ ]:
df = pd.read_csv('cpu_IO.csv', parse_dates=['Time'], index_col='Time')

df = df.dropna()
ax = df.plot(figsize=(20,10))
ax.set_ylabel('cpu io wait msec')
plt.show()


In [ ]:
df.plot(subplots=True,
        linewidth=1,
        layout=(5,2),
        figsize=(16,12),
        sharex=False,
        sharey=False)
plt.show()

In [ ]:
def running_average(x, order):
    current = x[:order].sum()
    running = []

    for i in range(order, x.shape[0]):
        current += x[i]
        current -= x[i-order]
        running.append(current/order)
    
    return np.array(running)

for feature in df.columns:
    trend = running_average(df[feature], 12)
    fig, ax = plt.subplots(1, 1)
    ax.plot(df.index, df[feature])
    ax.set_xlabel('Date')
    ax.set_ylabel(feature)
    ax.plot(df.index[12:], trend, label='Trend')
    ax.legend()

In [ ]:
ax = df.boxplot(fontsize=6, vert=False)
ax.set_xlabel('cpu io wait msec')
plt.show()

In [ ]:
# Pearson correlation matrix
corr_p = df[df.columns].corr(method='pearson')
print('\nPearson correlation matrix')
print(corr_p)

# Spearman correlation matrix'
corr_s = df[df.columns].corr(method='spearman')
print('\nSpearman correlation matrix')
print(corr_s)

In [ ]:
corr_mat = df.corr(method='pearson')
sns.heatmap(corr_mat)

In [ ]:
corr_mat_spearman = df.corr(method='spearman')
sns.heatmap(corr_mat_spearman)

In [ ]:
dict_decompositions = {}
df.names = df.columns
print(df.names)


In [ ]:
for feature in df.names:
    feature_decomposition = sm.tsa.seasonal_decompose(df[feature], period=43)
    dict_decompositions[feature] = feature_decomposition

dict_trend = {}
dict_seasonal = {}
dict_resid = {}

for feature in df.names:
    dict_trend[feature] = dict_decompositions[feature].trend  #TODo Check the period and where to insert!
    dict_seasonal[feature] = dict_decompositions[feature].seasonal
    dict_resid[feature] = dict_decompositions[feature].resid

trend_df = pd.DataFrame.from_dict(dict_trend)
seasonal_df = pd.DataFrame.from_dict(dict_seasonal)
resid_df = pd.DataFrame.from_dict(dict_resid)

Seasonal component plot

In [ ]:
seasonal_df.plot(subplots=True, layout=(5, 2),
          figsize=(17, 10),
          sharex=True,
          sharey=False)
plt.title('seasonal component of cpu io wait dataset')

trend component plot

In [ ]:
trend_df.plot(
    subplots=True,
    layout=(5,2),
    figsize=(17,10),
    sharex=True,
    sharey=False
)
plt.title('trend component of cpu io wait msec dataset')

resid component plot

In [ ]:
resid_df.plot(
    subplots=True,
    layout=(5,2),
    figsize=(17,10),
    sharex=True,
    sharey=False
)
plt.title('residual component of cpu io wait msec dataset')

# mixed plots per df in trend, seasonal and residual

In [ ]:
trend_df.plot()
plt.title('trend of the feature in the dataset')

seasonal_df.plot()
plt.title('seasonal of the feature in the dataset')

resid_df.plot()
plt.title('residual of the feature in the dataset')

# Cluster heatmap

In [ ]:
colonne_da_eliminare = ['cpu 8', 'cpu 9']
trend_df_clean = trend_df.drop(columns=colonne_da_eliminare)

trend_corr = trend_df_clean.dropna().corr(method='pearson')

# Customize the clustermap of the seasonality_corr
fig = sns.clustermap(trend_corr, annot=True, linewidth=0.5,figsize=(9,7))

plt.setp(fig.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
plt.setp(fig.ax_heatmap.xaxis.get_majorticklabels(), rotation=90)

In [ ]:
trend_corr_spearman = trend_df.corr(method='spearman')

fig = sns.clustermap(trend_corr, annot=True, linewidth=0.4,figsize=(9,7))

plt.setp(fig.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
plt.setp(fig.ax_heatmap.xaxis.get_majorticklabels(), rotation=90)